## Run this notebook

You can launch this notebook in the US GHG Center JupyterHub by clicking the link below.

[Launch in the US GHG Center JupyterHub (requires access)](https://hub.ghg.center/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2FUS-GHG-Center%2Fghgc-docs&urlpath=lab%2Ftree%2Fghgc-docs%2Fuser_data_notebooks%2Foco2geos-co2-daygrid-v10r_User_Notebook.ipynb&branch=main)
   

## Approach

1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the OCO-2 GEOS Column CO₂ Concentrations data product.
2. Pass the STAC item into the raster API `/collections/{collection_id}/items/{item_id}/tilejson.json` endpoint.
3. Using `folium.plugins.DualMap`, visualize two tiles (side-by-side), allowing time point comparison.
4. After the visualization, perform zonal statistics for a given polygon.

   

## About the Data

In July 2014, NASA successfully launched the first dedicated Earth remote sensing satellite to study atmospheric carbon dioxide (CO₂) from space. The Orbiting Carbon Observatory-2 (OCO-2) is an exploratory science mission designed to collect space-based global measurements of atmospheric CO₂ with the precision, resolution, and coverage needed to characterize sources and sinks (fluxes) on regional scales (≥1000 km). This dataset provides global gridded, daily column-averaged carbon dioxide (XCO₂) concentrations from January 1, 2015 - February 28, 2022. The data are derived from OCO-2 observations that were input to the Goddard Earth Observing System (GEOS) Constituent Data Assimilation System (CoDAS), a modeling and data assimilation system maintained by NASA’s Global Modeling and Assimilation Office (GMAO). Concentrations are measured in moles of carbon dioxide per mole of dry air (mol CO₂/mol dry) at a spatial resolution of 0.5° x 0.625°. Data assimilation synthesizes simulations and observations, adjusting modeled atmospheric constituents like CO₂ to reflect observed values. With the support of NASA’s Carbon Monitoring System (CMS) Program and the OCO Science Team, this dataset was produced as part of the OCO-2 mission which provides the highest quality space-based XCO₂ retrievals to date.

For more information regarding this dataset, please visit the [OCO-2 GEOS Column CO₂ Concentrations](https://earth.gov/ghgcenter/data-catalog/oco2geos-co2-daygrid-v10r) data overview page.

## Install the Required Libraries
Required libraries are pre-installed on the GHG Center Hub. If you need to run this notebook elsewhere, please install them with this line in a code cell:

%pip install requests folium rasterstats pystac_client pandas matplotlib --quiet

In [1]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt

## Querying the STAC API
First, we are going to import the required libraries. Once imported, they allow better executing a query in the GHG Center Spatio Temporal Asset Catalog (STAC) Application Programming Interface (API) where the granules for this collection are stored.

In [2]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# Please use the collection name similar to the one used in STAC collection.
# Name of the collection for OCO-2 GEOS Column CO₂ Concentrations. 
collection_name = "oco2geos-co2-daygrid-v10r"

In [3]:
# Fetching the collection from STAC collections using appropriate endpoint.
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'oco2geos-co2-daygrid-v10r',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/oco2geos-co2-daygrid-v10r/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/oco2geos-co2-daygrid-v10r'}],
 'title': 'OCO-2 GEOS Column CO₂ Concentrations v10r',
 'extent': {'spatial': {'bbox': [[-180.0, -90.0, 180.0, 90.0]]},
  'temporal': {'interval': [['2015-01-01T00:00:00+00:00',
     '2022-02-28T00:00:00+00:00']]}},
 'license': 'CC0-1.0',
 'renders': {'xco2': {'assets': ['xco2'],
   'nodata': 0,
   'rescale': [[412, 422]],
   'colormap_name': 'magma'},
  'dashboard': {'assets': ['xco2'],
   'nodata': 0,
   'rescale': [[412, 422]],
 

Examining the contents of our `collection` under the `temporal` variable, we see that the data is available from January 2015 to February 2022. By looking at the `dashboard:time density`, we can see that these observations are collected daily.

In [4]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [5]:
# Check total number of items available
number_of_items = get_item_count(collection_name)
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]
print(f"Found {len(items)} items")

Found 2615 items


In [6]:
# Examining the first item in the collection
items[0]

{'id': 'oco2geos-co2-daygrid-v10r-20220228',
 'bbox': [-180.3125, -90.25, 179.6875, 90.25],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/oco2geos-co2-daygrid-v10r'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/oco2geos-co2-daygrid-v10r'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/oco2geos-co2-daygrid-v10r/items/oco2geos-co2-daygrid-v10r-20220228'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/oco2geos-co2-daygrid-v10r/items/oco2geos-co2-daygrid-v10r-20220228/map?assets=xco2&nodata=0&rescale=412%2C422&colormap_name=magma',
   'rel': 'preview',
   'type': 'text/html'}],
 'assets': {'xco2': {'href': 's3://ghg

Below, we enter minimum and maximum values to provide our upper and lower bounds in `rescale_values`.

## Exploring Changes in Column-Averaged XCO₂ Concentrations Levels Using the Raster API

In this notebook, we will explore the temporal impacts of CO₂ emissions. We will visualize the outputs on a map using `folium.`

In [7]:
# To access the year value from each item more easily, this will let us query more explicitly by year and month (e.g., 2020-02)
items = {item["properties"]["datetime"]: item for item in items} 
asset_name = "xco2" #fossil fuel

In [8]:
# Fetching the min and max values for a specific item
rescale_values = {"max":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

Now, we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this twice, once for 2022-02-08 and again for 2022-01-27, so that we can visualize each event independently.

In [9]:
color_map = "magma"
oco2_1 = requests.get(
    f"{RASTER_API_URL}/collections/{items[list(items.keys())[0]]['collection']}/items/{items[list(items.keys())[0]]['id']}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
oco2_1

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/oco2geos-co2-daygrid-v10r/items/oco2geos-co2-daygrid-v10r-20220228/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=xco2&color_formula=gamma+r+1.05&colormap_name=magma&rescale=411.7429234611336%2C423.60419320175424'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.3125, -90.25, 179.6875, 90.25],
 'center': [-0.3125, 0.0, 0]}

In [10]:
oco2_2 = requests.get(
    f"{RASTER_API_URL}/collections/{items[list(items.keys())[1]]['collection']}/items/{items[list(items.keys())[1]]['id']}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
oco2_2

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/oco2geos-co2-daygrid-v10r/items/oco2geos-co2-daygrid-v10r-20220227/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=xco2&color_formula=gamma+r+1.05&colormap_name=magma&rescale=411.7429234611336%2C423.60419320175424'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.3125, -90.25, 179.6875, 90.25],
 'center': [-0.3125, 0.0, 0]}

## Visualizing Daily Column-Averaged XCO₂ Concentrations

### Setting Up an Interactive Dual Map for Visualizing CO₂ Layers from 2019 and 2020

In [11]:
# Set initial zoom and center of map for XCO₂ Layer
# Centre of map [latitude,longitude]
map_ = folium.plugins.DualMap(location=(34, -118), zoom_start=6)


map_layer_2020 = TileLayer(
    tiles=oco2_1["tiles"][0],
    attr="GHG",
    opacity=0.5,
)
map_layer_2020.add_to(map_.m1)

map_layer_2019 = TileLayer(
    tiles=oco2_2["tiles"][0],
    attr="GHG",
    opacity=0.5,
)
map_layer_2019.add_to(map_.m2)

# visualising the map
map_

### Analysis and Visualization of Maximum CO₂ Concentration Using Raster Data and Folium

In [12]:
import rasterio
import numpy as np
import folium
from folium import Marker, TileLayer
import folium.plugins

# Paso 1: Cargar el archivo TIFF y obtener la mayor concentración de CO₂
tif_path = 'oco2_GEOS_XCO2_L3CO2_day_B10206Ar_20220228.tif'

with rasterio.open(tif_path) as dataset:
    # Leer los datos como una matriz numpy
    band1 = dataset.read(1)

    # Encontrar el valor máximo (mayor concentración de CO₂)
    max_value = np.max(band1)
    max_index = np.unravel_index(np.argmax(band1), band1.shape)
    
    # Obtener las coordenadas geográficas del valor máximo
    max_coords = dataset.transform * (max_index[1], max_index[0])
    max_coords = [max_coords[1], max_coords[0]]  # Convertir a formato [lat, lon]

    print(f"La mayor concentración de CO₂ está en {max_coords} con un valor de {max_value}")

# Paso 2: Crear el mapa dual en folium
map_ = folium.plugins.DualMap(location=max_coords, zoom_start=6)

# Supongamos que ya tienes las capas de los mapas de CO₂ de 2020 y 2019
# Agregar las capas del 2020 y 2019 como en tu código original
map_layer_2020 = TileLayer(
    tiles=oco2_1["tiles"][0],  # Reemplaza oco2_1 con la variable correcta
    attr="GHG",
    opacity=0.5,
)
map_layer_2020.add_to(map_.m1)

map_layer_2019 = TileLayer(
    tiles=oco2_2["tiles"][0],  # Reemplaza oco2_2 con la variable correcta
    attr="GHG",
    opacity=0.5,
)
map_layer_2019.add_to(map_.m2)

# Paso 3: Añadir un marcador en el punto de mayor concentración de CO₂
Marker(location=max_coords, popup="Mayor concentración de CO₂").add_to(map_.m1)
Marker(location=max_coords, popup="Mayor concentración de CO₂").add_to(map_.m2)

# Visualizar el mapa
map_


La mayor concentración de CO₂ está en [39.75, 115.9375] con un valor de 423.60419320175424


### Identifying and Visualizing the Top Ten CO₂ Concentration Points Using Raster Data and Folium

In [13]:
import rasterio
import numpy as np
import folium
from folium import Marker, TileLayer
import folium.plugins
from rasterio.transform import rowcol
from geopy.distance import geodesic

# Función para calcular si dos puntos están a más de min_distance_km de distancia
def is_significantly_distant(coords1, coords2, min_distance_km=500):
    """ Verifica si dos coordenadas están a más de min_distance_km de distancia. """
    return geodesic(coords1, coords2).kilometers > min_distance_km

# Paso 1: Cargar el archivo TIFF y encontrar los diez valores más altos de CO₂
tif_path = 'oco2_GEOS_XCO2_L3CO2_day_B10206Ar_20220228.tif'

with rasterio.open(tif_path) as dataset:
    # Leer los datos como una matriz numpy
    band1 = dataset.read(1)
    mask_radius = 10  # número de píxeles a excluir alrededor de cada máximo
    locations = []  # Lista para almacenar las coordenadas de los máximos

    # Función para encontrar un nuevo máximo que esté significativamente distante de los anteriores
    def find_next_max(masked_band, existing_coords, dataset):
        while True:
            max_value = np.nanmax(masked_band)
            max_index = np.unravel_index(np.nanargmax(masked_band), masked_band.shape)
            max_coords = dataset.transform * (max_index[1], max_index[0])
            max_coords = [max_coords[1], max_coords[0]]  # Convertir a formato [lat, lon]

            # Verificar que esté suficientemente lejos de todas las ubicaciones existentes
            if all(is_significantly_distant(max_coords, coord) for coord in existing_coords):
                return max_value, max_coords, max_index
            else:
                # Enmascarar el área y continuar buscando
                masked_band[max_index[0] - mask_radius:max_index[0] + mask_radius,
                            max_index[1] - mask_radius:max_index[1] + mask_radius] = np.nan

    # Encontrar los diez máximos
    for i in range(10):
        max_value, max_coords, max_index = find_next_max(band1, [loc[1] for loc in locations], dataset)
        locations.append((max_value, max_coords, max_index))
        # Enmascarar el área alrededor del máximo encontrado
        band1[max_index[0] - mask_radius:max_index[0] + mask_radius,
              max_index[1] - mask_radius:max_index[1] + mask_radius] = np.nan

    # Mostrar las coordenadas de los máximos
    for idx, (value, coords, _) in enumerate(locations, start=1):
        print(f"Lugar {idx}: {coords} con valor {value}")

# Paso 2: Crear el mapa dual en folium
map_ = folium.plugins.DualMap(location=locations[0][1], zoom_start=6)

# Agregar las capas de 2020 y 2019 como en tu código original
map_layer_2020 = TileLayer(
    tiles=oco2_1["tiles"][0],  # Reemplaza oco2_1 con la variable correcta
    attr="GHG",
    opacity=0.5,
)
map_layer_2020.add_to(map_.m1)

map_layer_2019 = TileLayer(
    tiles=oco2_2["tiles"][0],  # Reemplaza oco2_2 con la variable correcta
    attr="GHG",
    opacity=0.5,
)
map_layer_2019.add_to(map_.m2)

# Paso 3: Añadir marcadores para los diez lugares con mayor concentración de CO₂
for idx, (value, coords, _) in enumerate(locations, start=1):
    Marker(location=coords, popup=f"Lugar {idx}: concentración de CO₂").add_to(map_.m1)
    Marker(location=coords, popup=f"Lugar {idx}: concentración de CO₂").add_to(map_.m2)

# Visualizar el mapa
map_


Lugar 1: [39.75, 115.9375] con valor 423.60419320175424
Lugar 2: [34.25, 119.6875] con valor 423.3151012158487
Lugar 3: [61.75, 100.3125] con valor 422.8237594361417
Lugar 4: [52.75, 164.0625] con valor 422.49004400218837
Lugar 5: [41.75, 122.1875] con valor 422.112199885305
Lugar 6: [78.75, 94.6875] con valor 422.03131670248695
Lugar 7: [47.75, 164.0625] con valor 421.86550126643857
Lugar 8: [44.25, 131.5625] con valor 421.81791286566295
Lugar 9: [60.25, 113.4375] con valor 421.6786692268215
Lugar 10: [58.75, 92.1875] con valor 421.5182525513228


### Analyzing and Visualizing the Highest CO₂ Emission in Africa Using Raster Data and Folium

In [14]:
import rasterio
import numpy as np
import folium
from folium import Marker, TileLayer
import folium.plugins
from geopy.distance import geodesic

# Paso 1: Cargar el archivo TIFF y encontrar la emisión más alta de CO₂ en África
tif_path = 'oco2_GEOS_XCO2_L3CO2_day_B10206Ar_20220228.tif'

with rasterio.open(tif_path) as dataset:
    # Leer los datos como una matriz numpy
    band1 = dataset.read(1)
    
    # Definir los límites geográficos de África
    def is_in_africa(coord):
        lat, lon = coord
        return -35 < lat < 37 and -20 < lon < 60

    # Inicializar variables para almacenar el máximo y las coordenadas
    max_value = -np.inf
    max_coords = None

    # Iterar sobre todos los píxeles
    for row in range(band1.shape[0]):
        for col in range(band1.shape[1]):
            value = band1[row, col]
            if value > 0:  # Considerar solo valores válidos
                coords = dataset.transform * (col, row)
                coords = [coords[1], coords[0]]  # Convertir a formato [lat, lon]
                if is_in_africa(coords) and value > max_value:
                    max_value = value
                    max_coords = coords

    # Mostrar el resultado
    if max_coords is not None:
        print(f"Emisión más alta en África: {max_value} en las coordenadas: {max_coords}")
    else:
        print("No se encontraron emisiones válidas en África.")

# Paso 2: Crear el mapa dual en folium
map_ = folium.plugins.DualMap(location=max_coords if max_coords is not None else [3.25, 11.5625], zoom_start=6)

# Agregar las capas de 2020 y 2019 como en tu código original
map_layer_2020 = TileLayer(
    tiles='https://earth.gov/ghgcenter/api/raster/collections/oco2geos-co2-daygrid-v10r/items/oco2geos-co2-daygrid-v10r-20220228/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=xco2&color_formula=gamma+r+1.05&colormap_name=magma&rescale=411.7429234611336%2C423.60419320175424',  # URL de 2020
    attr="GHG",
    opacity=0.5,
)
map_layer_2020.add_to(map_.m1)

map_layer_2019 = TileLayer(
    tiles='https://earth.gov/ghgcenter/api/raster/collections/oco2geos-co2-daygrid-v10r/items/oco2geos-co2-daygrid-v10r-20190228/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=xco2&color_formula=gamma+r+1.05&colormap_name=magma&rescale=411.7429234611336%2C423.60419320175424',  # URL de 2019
    attr="GHG",
    opacity=0.5,
)
map_layer_2019.add_to(map_.m2)

# Paso 3: Añadir un marcador para la emisión más alta
if max_coords is not None:
    Marker(location=max_coords, popup=f"Emisión más alta: {max_value}").add_to(map_.m1)
    Marker(location=max_coords, popup=f"Emisión más alta: {max_value}").add_to(map_.m2)

# Añadir el marcador para la coordenada específica (3.25, 11.5625)
specific_coords = [3.25, 11.5625]
Marker(location=specific_coords, popup="Coordenada específica: 3.25, 11.5625").add_to(map_.m1)
Marker(location=specific_coords, popup="Coordenada específica: 3.25, 11.5625").add_to(map_.m2)

# Visualizar el mapa
map_


Emisión más alta en África: 421.0884289932437 en las coordenadas: [3.25, 11.5625]
